In [2]:
!pip install pymupdf pandas numpy openai

   ---------------------------------------- 0.0/3.4 MB ? eta -:--:--
   - -------------------------------------- 0.1/3.4 MB 3.6 MB/s eta 0:00:01
   --- ------------------------------------ 0.3/3.4 MB 3.5 MB/s eta 0:00:01
   ----- ---------------------------------- 0.5/3.4 MB 3.8 MB/s eta 0:00:01
   ------- -------------------------------- 0.7/3.4 MB 4.1 MB/s eta 0:00:01
   ----------- ---------------------------- 1.0/3.4 MB 4.5 MB/s eta 0:00:01
   ---------------- ----------------------- 1.4/3.4 MB 5.7 MB/s eta 0:00:01
   --------------------- ------------------ 1.8/3.4 MB 6.1 MB/s eta 0:00:01
   ------------------------ --------------- 2.1/3.4 MB 6.1 MB/s eta 0:00:01
   ---------------------------- ----------- 2.4/3.4 MB 6.2 MB/s eta 0:00:01
   -------------------------------- ------- 2.8/3.4 MB 6.4 MB/s eta 0:00:01
   ------------------------------------ --- 3.1/3.4 MB 6.4 MB/s eta 0:00:01
   ---------------------------------------  3.4/3.4 MB 6.6 MB/s eta 0:00:01
   ----------------

In [3]:
import fitz

In [4]:
raw = fitz.open('data-test-3.pdf')

pars = []

for page in raw:
    pars.extend([b[4].replace('\n', ' ') for b in page.get_text("blocks") if b[4] and len(b[4]) > 6])

In [5]:
def group_paragraphs(pars, min_l=50):
    g_pars = []
    curr = ""

    for p in pars:
        curr += ("   " if curr else "") + p.strip()
        if len(curr) > min_l:
            g_pars.append(curr)
            curr = ""
    if curr:
        g_pars.append(curr)
    
    return g_pars

In [6]:
g_pars = group_paragraphs(pars)

In [7]:
import numpy as np
import pandas as pd

In [8]:
df = pd.DataFrame(g_pars, columns=['Text'])

In [9]:
df = df.shift(2, fill_value="") + df.shift(1, fill_value="") + df + df.shift(-1, fill_value="") + df.shift(-2, fill_value="")

In [13]:
from openai import AzureOpenAI
import os

os.environ["AZURE_OPENAI_KEY"] = "7e93421f46cd4680831023addcb0f42d"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://francecentral-openai.openai.azure.com"
os.environ["AZURE_OPENAI_DeploymentId"] = "ada-002"
client = AzureOpenAI(
    api_key = os.getenv("AZURE_OPENAI_KEY"),
    api_version = "2023-05-15",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

def generate_embeddings(text, model="ada-002"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

df['embed'] = df["Text"].apply(lambda x : generate_embeddings (x, model = "ada-002")) 

In [14]:
df.head()

,Text,embed
0,BANQUE NATIONALE DE PARIS Société anonyme ...,"[-0.03115769848227501, -0.02030486986041069, 0..."
1,BANQUE NATIONALE DE PARIS Société anonyme ...,"[-0.027154553681612015, -0.02024681679904461, ..."
2,BANQUE NATIONALE DE PARIS Société anonyme ...,"[-0.031345997005701065, -0.021061424165964127,..."
3,"Siège social : 16, boulevard des Italiens, 750...","[-0.028570761904120445, -0.01988103613257408, ..."
4,R.C.S. PARIS B 662 042 449 fiche d'informati...,"[-0.022284362465143204, -0.01317229587584734, ..."


In [24]:
import pandas as pd
import numpy as np
import json
import os
from openai import AzureOpenAI

class VegaAnalyst:
  def __init__(self, system_prompt):
    os.environ["AZURE_OPENAI_KEY"] = "7e93421f46cd4680831023addcb0f42d"
    os.environ["AZURE_OPENAI_ENDPOINT"] = "https://francecentral-openai.openai.azure.com"
    os.environ["AZURE_OPENAI_DeploymentId"] = "gpt-4-turbo"
    self.client = AzureOpenAI(
      api_key = os.getenv("AZURE_OPENAI_KEY"),  
      api_version = "2023-05-15",
      azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

    self.embeddings = df
    self.system_prompt = system_prompt

  def _cosine_similarity(self, a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
  
  def _get_embedding(self, text, model="ada-002"): # model = "deployment_name"
    return self.client.embeddings.create(input = [text], model=model).data[0].embedding

  def search_paragraphs(self, _embeddings: pd.DataFrame, user_query, top_n=4):
    embedding = self._get_embedding(
        user_query,
        model="ada-002" # model should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
    )
    embeddings = _embeddings.copy()
    embeddings["similarities"] = embeddings["embed"].apply(lambda x: self._cosine_similarity(x, embedding))

    res = (
        embeddings.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    return res

  def generate_answer(self, prompt, system_prompt):
    res = self.client.chat.completions.create(
      model=os.getenv("AZURE_OPENAI_DeploymentId"),
      response_format={"type": "json_object"},
      messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
      ]
    ).choices[0].message
    
    return res.content

  def analyse(self, prompt):
    res = self.search_paragraphs(self.embeddings, prompt, top_n=2)
    sys_prompt = self.system_prompt + "\n".join(res["Text"])
    answer = self.generate_answer(prompt, sys_prompt)
    return ({"answer": answer, "docs": res })
  

In [11]:
import os
import json

root = "./streamlitapp/pdc_content"

walk = [dir for dir in os.walk(root)]
cats = []
for dir in walk[0][1]:
    with open(f'{root}/{dir}/info.json', encoding="utf-8") as f:
        d = json.load(f)
        cats.append(d)

pdcs = []

for i, cat in enumerate(cats):
    code = cat["code"]
    jsons = [f for f in walk[i+1][2] if f != "info.json"]
    for fname in jsons:
        with open(f'{root}/{code}/{fname}', encoding="utf-8") as f:
            dataf = json.load(f)
            pdcs.append(dataf)

In [18]:
system_prompt = """Objectif Principal : Tu es un expert de l'Authorité de Contrôle Prudentiel de Résolution rattaché à la banque de France. Ta mission est de juger de la conformité d'une pratique relativement à un corpus de textes règlementaires et normatifs qui t'es fourni. Ce que tu écris doit pouvoir être présenté dans des rapports officiels. Tu n'es pas un agent de conversation. 

1. Accès aux Informations :
- Tu dois extraire de manière précise les informations pertinentes des documents normatifs pour juger de la conformité d'une pratique.
- Tu dois restituer le contexte entourant une information pour une meilleure compréhension. Fais bien attention, à séparer la citation du texte normatif des informations contextuelles. Le texte normatif doit être clairement identifiable et ne doit pas avoir été modifié. 

2. Pré-évaluation de la conformité :
- Tu dois utiliser la méthodologie décrite dans les points de contrôle afin de vérifier la conformité.
- Pour pré-évaluer la conformité, tu dois aussi te baser sur les exemples de conformité et les exemples de non-conformité donnés dans les points de contrôle.

3. Gestion des Erreurs et Ambiguïtés :
- Tu dois gérer les situations où une requête est ambiguë ou incomplète en demandant des clarifications.
- Si l’utilisateur fait une faute de frappe, corrige-la en citant la correction et réponds à la question posée par la phrase corrigée. 
- Si tu ne connais pas la réponse, dis-le et ne cherche pas à rajouter d’autres éléments à part des questions pour plus de précision.
- Si tu lis une abréviation dans un document ou si tu lis une abréviation dans la phrase de l’utilisateur et que tu ne comprends pas l’abréviation, cite l’abréviation et demande à l’utilisateur de la définir. 

4. Mises à Jour Légales :
- Tu dois informer les utilisateurs des modifications récentes dans la législation financière en précisant les dates. 

5. Résultats :
- Une fois que tu as analysé la conformité de la pratique, tu dois fournir une réponse dans un fichier JSON. Le fichier doit être structuré ainsi : 
{
"category" : "émission de dette",
"code" : "5mat"
"name" : "AT1 Own Funds Instruments  Monitoring",
"desc" : "Les dettes doivent être émises publiquement et être disponibles à tout le monde",
"base_leg" : [
"1. Article 1", 
"2. Article 2 du Code monétaire"],
"practise_summary" : "<insère la pratique soumise en entrée>",
"summary": "<insère ta réponse en langage naturel et explique ton raisonnement>",
"conform" : "oui / non / nsp / N/A",
"procedure" : "En cas de non-conformité, il faut informer l’établissement de la non-conformité et demander les éléments de remédiation dans un délai de 3 mois / Aucune procédure à suivre / Je ne parviens pas à juger de la conformité de cette pratique, une analyse humaine est peut-être requise, sinon veuillez reformuler la demande / Cette pratique semble ne pas être en relation avec mes compétences et donc mon analyse n'est pas applicable"
}
- Si un champ du JSON n'est pas applicable, remplis le avec "N/A"
- Si tu n'est pas sûr de la conformité de la procédure vis à vis du point de contrôle, remplis le champ "conform" avec la valeur "nsp"

6. Procédure à évaluer :
- Voici des paragraphes de la procédure qui sont en rapport avec le point de contrôle, dont tu dois évaluer la conformité :
"""

In [16]:
pdcs

[{'category_name': "Emission d'instruments de fonds propres",
  'name': 'Tier 2 Own Funds instrument characteristics monitoring  ',
  'code': '5mat',
  'desc': 'Please confirm that the instrument has a maturity of more than five years.',
  'base_leg': ['Article 63(g) Regulation (EU) No 575/2013'],
  'methodology': ['Search in the document the maturity date of the issuance ',
   'If there is no maturity date, the instrument is perpetual',
   'If the instrument is perpetual, then the instrument is compliant',
   'If there is a maturity date, then the instrument is not compliant'],
  'conform_example': '',
  'non_conform_example': '',
  'proc': 'Contact the Supervised Entity to organize a meeting.  '},
 {'category_name': "Emission d'instruments de fonds propres",
  'name': 'Verification of perpetual maturity',
  'code': 'perp',
  'desc': 'Please confirm that the instruments are perpetual.  ',
  'base_leg': ['Article  52(1)(g) Regulation (EU) No 575/2013'],
  'methodology': ['Please search

In [25]:
analyst = VegaAnalyst(system_prompt)

res = analyst.analyse(f"Analyse la conformité du texte qui t'es fourni vis à vis du point de contrôle suivant :\n{pdcs[0]}")

In [27]:
res["answer"]

'{\n"Catégorie" : "Emission d\'instruments de fonds propres",\n"Numéro du point de contrôle" : "5mat",\n"Intitulé du point de contrôle" : "Tier 2 Own Funds instrument characteristics monitoring",\n"Base légale" : [\n"Article 63(g) Regulation (EU) No 575/2013"\n],\n"Pratique" : "émission de F 2.000.000.000 représentés par 400.000 titres subordonnés à durée indéterminée de F 5.000 nominal comportant chacun un bon de souscription à une obligation.",\n"Conformité" : "oui",\n"Procédure à suivre" : "Aucune procédure à suivre",\n"Résumé": "La pratique soumise fait état d\'une émission de titres subordonnés à durée indéterminée. Selon la méthodologie fournie, si un instrument financier possède une durée perpétuelle, alors il est en conformité avec l\'article 63(g) du Règlement (UE) No 575/2013 concernant les caractéristiques des instruments du Tier 2 des fonds propres qui stipule que l\'instrument doit avoir une maturité supérieure à cinq ans. La durée indéterminée signifie que ces titres sont